In [1]:
import numpy as np
import pyamg
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression

from scipy.sparse.linalg import eigsh

import scipy
%matplotlib notebook

In [2]:
grid_sizes = [ 5, 10,
              15, 20, 
              25, 30, 
              35, 40, 
              45, 50,
              55, 60,
              65, 70,
              75, 80,
              85, 90,
              95, 100] #20 total

scales = [1e-7, 5e-7,
          1e-6, 5e-6, 
          1e-5, 5e-5, 
          1e-4, 5e-4, 
          1e-3, 5e-3,]

In [3]:
runs_per = 10

def cg(A, b):
    return pyamg.krylov.cg(A, b, tol=1e-8, maxiter=min(A.shape[0],1000))[0]

def mg(A, b):
    return pyamg.ruge_stuben_solver(A).solve(b, tol=1e-8)

def sd(A, b):
    return pyamg.krylov.steepest_descent(A, b, tol=1e-8)[0]

def gmres(A, b):
    return pyamg.krylov.gmres(A, b, tol=1e-8, maxiter=min(A.shape[0],1000))[0]

methods = [("CG",cg), ("MG",mg), ("SD", sd), ("GMRES", gmres)]

In [5]:
max_k = 5
max_r = 5

results_df3 = pd.DataFrame(columns=['Method','N','sigma','run','side','k','lambda',
                                    'norm((x-y)/sigma+v_k/lambda_k)','norm(x-y)'])

for grid_i in range(1,len(grid_sizes),2):
    temp_data = []
    print("Grid: ", grid_i)
    n = grid_sizes[grid_i]

    A = pyamg.gallery.poisson((n,n), format='csr')
    
    eigs = {}
    eigs['large'] = eigsh(A, k=max_k, which='LM')
    eigs['small'] = eigsh(A, k=max_k, which='SM')

    for scale_i in range(0,len(scales),2):
        print("    Scale: ", scale_i)
        
        for run_i in range(runs_per):

            for k in range(max_k):
                b = np.random.rand(A.shape[0])
                E_large = eigs['large'][1][:,k] * scales[scale_i]
                E_small = eigs['small'][1][:,k] * scales[scale_i]

                b_large = b + E_large
                b_small = b + E_small

                for method_i in range(len(methods)):
                    name, solver = methods[method_i]
                    x = solver(A, b)
                    x_large = solver(A, b_large)
                    x_small = solver(A, b_small)

                    temp_data.append({
                        'Method':name,
                        'N':A.shape[0],
                        'sigma':scales[scale_i],
                        'run':run_i,
                        'side':'large',
                        'k':k,
                        'lambda':eigs['large'][0][k],
                        'norm((x-y)/sigma+v_k/lambda_k)':np.linalg.norm((x-x_large)/scales[scale_i]+E_large/eigs['large'][0][k]),
                        'norm(x-y)':np.linalg.norm(x-x_large),
                    })

                    temp_data.append({
                        'Method':name,
                        'N':A.shape[0],
                        'sigma':scales[scale_i],
                        'run':run_i,
                        'side':'small',
                        'k':k,
                        'lambda':eigs['small'][0][k],
                        'norm((x-y)/sigma+v_k/lambda_k)':np.linalg.norm((x-x_small)/scales[scale_i]+E_small/eigs['small'][0][k]),
                        'norm(x-y)':np.linalg.norm(x-x_small),
                    })

    results_df3 = results_df3.append(temp_data)

Grid:  1
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  3
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  5
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  7
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  9
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  11
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  13
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  15
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  17
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  19
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8


In [7]:
results_df3.to_csv('Poisson_Eigen_b.csv')

In [21]:
method = 'MG'
side = 'large'

results_method_side = results_df3[(results_df3['side']==side) & (results_df3['Method']==method)]
results_method_side['norm(x-y)']/results_method_side['sigma']-1/results_method_side['lambda']

2      -1.621027e-09
10      6.948328e-11
18     -5.534498e-09
26     -1.545886e-09
34     -5.983873e-09
            ...     
1962   -1.036890e-11
1970    1.731670e-12
1978    1.882877e-11
1986    1.856792e-11
1994    4.737555e-11
Length: 2500, dtype: float64

In [22]:
max_k=10

for grid_i in range(1,len(grid_sizes),2):
    temp_data = []
    print("Grid: ", grid_i)
    n = grid_sizes[grid_i]

    A = pyamg.gallery.poisson((n,n), format='csr')
    
    eigs = {}
    eigs['large'] = eigsh(A, k=max_k, which='LM')
    eigs['small'] = eigsh(A, k=max_k, which='SM')

    for scale_i in range(0,len(scales),2):
        print("    Scale: ", scale_i)
        
        for run_i in range(runs_per):

            for k in range(5,max_k):
                b = np.random.rand(A.shape[0])
                E_large = eigs['large'][1][:,k] * scales[scale_i]
                E_small = eigs['small'][1][:,k] * scales[scale_i]

                b_large = b + E_large
                b_small = b + E_small

                for method_i in range(len(methods)):
                    name, solver = methods[method_i]
                    x = solver(A, b)
                    x_large = solver(A, b_large)
                    x_small = solver(A, b_small)

                    temp_data.append({
                        'Method':name,
                        'N':A.shape[0],
                        'sigma':scales[scale_i],
                        'run':run_i,
                        'side':'large',
                        'k':k,
                        'lambda':eigs['large'][0][k],
                        'norm((x-y)/sigma+v_k/lambda_k)':np.linalg.norm((x-x_large)/scales[scale_i]+E_large/eigs['large'][0][k]),
                        'norm(x-y)':np.linalg.norm(x-x_large),
                    })

                    temp_data.append({
                        'Method':name,
                        'N':A.shape[0],
                        'sigma':scales[scale_i],
                        'run':run_i,
                        'side':'small',
                        'k':k,
                        'lambda':eigs['small'][0][k],
                        'norm((x-y)/sigma+v_k/lambda_k)':np.linalg.norm((x-x_small)/scales[scale_i]+E_small/eigs['small'][0][k]),
                        'norm(x-y)':np.linalg.norm(x-x_small),
                    })

    results_df3 = results_df3.append(temp_data)

Grid:  1
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  3
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  5
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  7
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  9
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  11
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  13
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  15
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  17
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8
Grid:  19
    Scale:  0
    Scale:  2
    Scale:  4
    Scale:  6
    Scale:  8


In [23]:
results_df3.to_csv('Poisson_Eigen_b.csv')